In [3]:
import numpy as np 
import pandas as pd

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [4]:
imdb_data=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

pd.set_option('display.max_colwidth', None)  
imdb_data.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

In [5]:
pd.set_option('display.max_colwidth', 50)  

In [6]:
len(imdb_data)

50000

In [7]:
# Check for null
imdb_data.isna().sum()

review       0
sentiment    0
dtype: int64

In [8]:
# Check for duplicates
imdb_data.duplicated().sum()

418

In [9]:
#418 duplicate rows, lets drop the duplicated data
imdb_data.drop_duplicates(inplace=True)
print(len(imdb_data))

49582


In [10]:
# Checking unique values & class imbalance in target 
imdb_data['sentiment'].value_counts()

sentiment
positive    24884
negative    24698
Name: count, dtype: int64

In [11]:
# Convert sentiment to numeric value
sentiment_mapping = {'positive': 1, 'negative': 0}

# Map the values using the defined mapping
imdb_data['sentiment'] = imdb_data['sentiment'].map(sentiment_mapping)

## Data Cleaning

In [12]:
### Convert to lower case
imdb_data['review']=imdb_data['review'].str.lower()
imdb_data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. <br /><br />the...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [13]:
# Remove tags
def remove_tag(s):
    pattern=re.compile(r'<[^>]+>')
    return re.sub(pattern, '', s)

imdb_data['review']=imdb_data['review'].apply(remove_tag)
imdb_data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [14]:
## Check for URL
# def contains_url(text):
#     pattern = re.compile(r'https?://\S+|www\.\S+')
#     matches = pattern.findall(r'', text)
#     return len(matches) > 0

In [15]:
# Remove Punctuations
exclude =  string.punctuation
def remove_punc(s):
    return s.translate(str.maketrans('', '', exclude))

imdb_data['review']=imdb_data['review'].apply(remove_punc)
imdb_data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [16]:
# Remove stop words
def remove_stop_words(words):
    en_stop_words=set(stopwords.words('english'))
    filtered_review = [word for word in words if word not in en_stop_words]
    return filtered_review

def stemming(words):
    porter_stemmer = PorterStemmer()
    stemmed_words = [porter_stemmer.stem(word) for word in words]
    return stemmed_words

def filter_words(review):
    words = word_tokenize(review)
    filtered_review=remove_stop_words(words)
    stemmed_words=stemming(filtered_review)
    return ' '.join(stemmed_words)

imdb_data['review']=imdb_data['review'].apply(filter_words)
imdb_data.head()

,review,sentiment
0,one review mention watch 1 oz episod youll hoo...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic there famili littl boy jake think there ...,0
4,petter mattei love time money visual stun film...,1


## Checking the vocabulary 

In [19]:
merged_row = imdb_data[['review']].apply(lambda w: ','.join(w.astype(str)))
merged_row

review    one review mention watch 1 oz episod youll hoo...
dtype: object